# Catalyst Cooperative Jupyter Notebook Template
This notebook lays out a standard format and some best practices for creating interactive / exploratory notebooks which can be relatively easily shared between different PUDL users, and turned into reusable Python modules for integration into our underlying Python packages.

## Begin with a narrative outline
Each notebook should start with a brief explanation (in Markdown) explaining the purpose of the analysis, and outlining the different stages / steps which are taken to accomplish the analysis.
As the analysis develops, you can add new sections or details to this section.

## Notebooks should be runnable
Insofar as it's possible, another PUDL user who has cloned the repository that the notebook is part of should be able to update their `pudl-dev` conda environment, open the notebook, and run all cells successfully.
If there are required data or other prerequisites that the notebook cannot manage on its own -- like a file that needs to be downloaded by hand and placed in a particular location -- those steps should be laid out clearly at the beginning of the notebook.

## Avoid Troublesome Elements

### Don't hardcode passwords or access tokens
Most of our work is done in public Github repositories.
No authentication information should ever appear in a notebook.
These values can be stored in environment variables on your local computer.

### Do not hardocde values, especially late in the notebook
If the analysis depends on particular choices of input values, those should be called out explicitly at the beginning of the notebook.
(NB: We should explore ways to parameterize notebooks, [papermill](https://papermill.readthedocs.io/en/latest/) is one tool that does this).

### Don't hardcode absolute file paths
If anyone is going to be able to use the notebook, the files it uses will need to be stored somewhere that makes sense on both your and other computers.
We assume that anyone using this template has the PUDL package installed, and has a local PUDL data management environment set up.
  * Input data that needs to be stored on disk and accessed via a shared location should be saved under `<PUDL_IN>/data/local/<data_source>/`.
  * Intermediate saved data products (e.g. a pickled result of a computationally intensive process) and results should be saved to a location relative to the notebook, and within the directory hierarchy of the repository that the notebook is part of.
  
### Don't require avoidable long-running computations
Consider persisting to disk the results of computations that take more than a few minutes, if the outputs are small enough to be checked into the repository and shared with other users.
Only require the expensive computation to be run if this pre-computed output is not available.

### Don't litter
Don't leave lots of additional code laying around, even commented out, "just in case" you want to look at it again later.
Notebooks need to be relatively linear in the end, even though the thought processes and exploratory analyses that generate them may not be.
Once you have a working analysis, either prune those branches, or encapsulate them as options within functions.

### Don't load unneccesary libraries
Only import libraries which are required by the notebook, to avoid unnecessary dependencies.
If your analysis requires a new library that isn't yet part of the shared `pudl-dev` environment, add it to the `devtools/environment.yml` file so that others will get it when they update their environment.

## Related Resources:
Lots of these guidelines are taken directly from Emily Riederer's post: [RMarkdown Driven Development](https://emilyriederer.netlify.app/post/rmarkdown-driven-development/).
For more in depth explanation of the motivations behind this layout, do go check it out!

# Import Libraries
* Because it's very likely that you will be editing the PUDL Python packages or your own local module under development while working in the notebook, use the %autoreload magic with autoreload level 2 to ensure that any changes you've made in those files are always reflected in the code that's running in the notebook.
* Put all import statements at the top of the notebook, so everyone can see what its dependencies are up front, and so that if an import is going to fail, it fails early, before the rest of the notebook is run.
* Try to avoid importing individual functions and classes from deep within packages, as it may not be clear to other users where those elements came from, later in the notebook, and also to minimize the chance of namespace collisions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

# Configure Display Parameters

In [3]:
sns.set()
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,4)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Use Python Logging facilities
* Using a logger from the beginning will make the transition into the PUDL package easier.
* Creating a logging handler here will also allow you to see the logging output coming from PUDL and other underlying packages.

In [4]:
# logger=logging.getLogger()
# logger.setLevel(logging.INFO)
# handler = logging.StreamHandler(stream=sys.stdout)
# formatter = logging.Formatter('%(message)s')
# handler.setFormatter(formatter)
# logger.handlers = [handler]

# Define Functions
In many cases, the eventual product of a notebook analysis is going to be the creation of new, reusable functions that are integrated into the underlying PUDL code. You should begin the process of accumulating and organizing those functions as soon as you notice repeated patterns in your code.
* Functions should be used to encapsulate any potentially reusable code.
* Functions should be defined immediately after the imports, to avoid accidental dependence on zombie variables that are defined further down in the code
* While they may evolve over time, having brief docstrings explaining what they do will help others understand them.
* If there's a particular type of plot or visualization that is made repeatedly in the notebook, it's good to parameterize and functionalize it here too, so that as you refine the presentation of the data and results, those improvements can be made in a single place, and shown uniformly throughout the notebook.
* As these functions mature and become more general purpose tools, you will probably want to start migrating them into their own local module, under a `src` directory in the same directory as the notebook. You will want to import this module 

## Dummy EIA 861 ETL

In [5]:
def test_etl_eia(eia_inputs, pudl_settings):
    """
    This is a dummy function that runs the first part of the EIA ETL
    process -- everything up until the entity harvesting begins. For
    use in this notebook only.

    """
    eia860_tables = eia_inputs["eia860_tables"]
    eia860_years = eia_inputs["eia860_years"]
    eia861_tables = eia_inputs["eia861_tables"]
    eia861_years = eia_inputs["eia861_years"]
    eia923_tables = eia_inputs["eia923_tables"]
    eia923_years = eia_inputs["eia923_years"]

    # generate CSVs for the static EIA tables, return the list of tables
    #static_tables = _load_static_tables_eia(datapkg_dir)

    # Extract EIA forms 923, 860
    eia860_raw_dfs = pudl.extract.eia860.Extractor().extract(eia860_years, testing=True)
    eia861_raw_dfs = pudl.extract.eia861.Extractor().extract(eia861_years, testing=True)
    eia923_raw_dfs = pudl.extract.eia923.Extractor().extract(eia923_years, testing=True)

    # Transform EIA forms 860, 861, 923
    eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
    eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
    eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

    # create an eia transformed dfs dictionary
    eia_transformed_dfs = eia860_transformed_dfs.copy()
    eia_transformed_dfs.update(eia861_transformed_dfs.copy())
    eia_transformed_dfs.update(eia923_transformed_dfs.copy())

    # convert types..
    eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

    return eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs

# Define Notebook Parameters
If there are overarching parameters which determine the nature of the analysis -- which US states to look at, which utilities are of interest, a particular start and end date -- state those clearly at the beginning of the analysis, so that they can be referred to by the rest of the notebook and easily changed if need be.
* If the analysis depends on local (non-PUDL managed) datasets, define the paths to those data here.
* If there are external URLs or other resource locations that will be accessed, define those here as well.
* This is also where you should create your `pudl_settings` dictionary and connections to your local PUDL databases

In [6]:
EIA861_YEARS = list(range(2001, 2019))
pudl_settings = pudl.workspace.setup.get_defaults()
display(pudl_settings)

ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
display(ferc1_engine)

pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
display(pudl_engine)

pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)


# Is there other external data you need to pull in?
# If so, put it in a (relatively) standard place on the filesystem.
my_new_data_url = "https://mynewdata.website.gov/path/to/new/data.csv"
my_new_datadir = pathlib.Path(pudl_settings["data_dir"]) / "local/new_data_source"

# Store API keys and other secrets in environment variables
# and read them in here, if needed:
# API_KEY_EIA = os.environ["API_KEY_EIA "]
# API_KEY_FRED = os.environ["API_KEY_FRED "]

{'pudl_in': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'data_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/data',
 'settings_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/settings',
 'pudl_out': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'sqlite_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite',
 'parquet_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/parquet',
 'datapkg_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/datapkg',
 'notebook_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/notebook',
 'ferc1_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite'}

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite)

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite)

# Load Data
* Given the above parameters and functions, it should now be possible to load the required data into local variables for further wrangling, analysis, and visualization.
* If the data is not yet present on the machine of the person running the notebook, this step should also acquire the data from its original source, and place it in the appropriate location under `<PUDL_IN>/data/local/`.
* If there are steps which have to be done manually here, put them first so that they fail first if the user hasn't read the instructions, and they can fix the situation before a bunch of other work gets done. Try to minimize the amount of work in the filesystem that has to be done manually though.
* If this process takes a little while, don't be shy about producing `logging` output.
* Using the `%%time` cell magic can also help users understand which pieces of work / data acquisition are hard:

## EIA 861 (2010-2018)
* Not yet fully integrated into PUDL
* Post-transform harvesting process isn't compatible w/ EIA 861 structure
* Only getting the `sales_eia861`, `balancing_authority_eia861`, and `service_territory_eia861` tables

### Already Transformed EIA 861 DataFrames

In [373]:
%%time
eia_inputs = {
    "eia860_years": [],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": EIA861_YEARS,
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}
eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs = test_etl_eia(eia_inputs=eia_inputs, pudl_settings=pudl_settings)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


CPU times: user 2min 16s, sys: 4.74 s, total: 2min 20s
Wall time: 2min 26s


# Sanity Check Data
If there's any validation that can be done on the data which you've loaded to flag if/when it is inappropriate for the analysis that follows, do it here. If you find the data is unusable, use `assert` statements or `raise` Exceptions to stop the notebook from proceeding, and indicate what the problem is.

In [620]:
# SHOULD ALSO MAKE REVENUE A NORMALIZED COLUMN??

In [374]:
test_df = eia_transformed_dfs['operational_data_eia861'].copy()
raw_df = eia861_raw_dfs['operational_data_eia861'].copy()

In [376]:
test_df

,utility_id_eia,state,nerc_region,report_date,consumed_by_facility_mwh,consumed_by_respondent_without_charge_mwh,data_observed,entity_type,exchange_energy_delivered_mwh,exchange_energy_received_mwh,furnished_without_charge_mwh,net_generation_mwh,net_power_exchanged_mwh,net_wheeled_power_mwh,retail_sales_mwh,revenue_from_credits_or_adjustments,revenue_from_delivery_customers,revenue_from_other,revenue_from_retail_sales,revenue_from_sales_for_resale,revenue_from_transmission,revenue_total,sales_for_resale_mwh,summer_peak_demand_mw,total_disposition_mwh,total_energy_losses_mwh,total_sources_mwh,transmission_by_other_losses_mwh,unbundled_revenues,utility_name_eia,wheeled_power_delivered_mwh,wheeled_power_received_mwh,wholesale_power_purchases_mwh,winter_peak_demand_mw
0,20139,IN,ECAR,2001-01-01,NaN,NaN,True,Municipal,NaN,NaN,NaN,NaN,0.0,0.0,159499.0,NaN,NaN,78000.0,8489000.0,NaN,NaN,8567000.0,NaN,37.0,168288.0,8789.0,168288.0,NaN,NaN,City of Washington,NaN,NaN,168288.0,26.0
1,7711,OH,MAAC,2001-01-01,NaN,NaN,True,Power Marketer,NaN,NaN,NaN,NaN,0.0,0.0,537195.0,NaN,NaN,NaN,25772000.0,NaN,NaN,25772000.0,NaN,NaN,575793.0,38598.0,575793.0,NaN,NaN,"GPU Advanced Resources, Inc",NaN,NaN,575793.0,NaN
2,21260,GA,UNK,2001-01-01,34551.0,NaN,True,Other,NaN,NaN,NaN,87.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,34551.0,NaN,34551.0,NaN,NaN,191 Peachtree Associates,NaN,NaN,34464.0,NaN
3,21156,NY,NPCC,2001-01-01,NaN,NaN,True,Cooperative,NaN,NaN,NaN,NaN,0.0,0.0,166498.0,NaN,NaN,NaN,16768000.0,NaN,NaN,16768000.0,NaN,39.0,169120.0,2622.0,169120.0,NaN,NaN,"1st Rochdale Cooperative Group,Ltd",NaN,NaN,169120.0,28.0
4,21152,CA,WSCC,2001-01-01,NaN,NaN,True,Power Marketer,NaN,NaN,NaN,NaN,0.0,0.0,6264.0,NaN,NaN,4000.0,472000.0,NaN,NaN,476000.0,NaN,NaN,6264.0,NaN,6264.0,NaN,NaN,3 Phases Electrical Consulting,NaN,NaN,6264.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52647,61823,CT,UNK,2018-01-01,NaN,NaN,True,Behind the Meter,0.0,0.0,NaN,147160.0,0.0,0.0,147160.0,0.0,0.0,0.0,11507900.0,0.0,0.0,11507900.0,0.0,NaN,147160.0,NaN,147160.0,NaN,NaN,"Greenskies Renewable Energy, LLC",NaN,NaN,NaN,NaN
52648,61858,CA,WECC,2018-01-01,NaN,0.0,True,Community Choice Aggregator,0.0,0.0,0.0,0.0,0.0,0.0,31118.0,0.0,0.0,0.0,2518700.0,0.0,0.0,2518700.0,0.0,20.8,39827.0,8709.0,39827.0,0.0,NaN,San Jose Clean Energy,0.0,0.0,39827.0,24.8
52649,62107,CA,UNK,2018-01-01,NaN,NaN,True,Municipal,0.0,0.0,NaN,NaN,0.0,0.0,39095.0,NaN,NaN,NaN,4482100.0,NaN,NaN,4482100.0,NaN,7.6,41069.0,1974.0,41069.0,NaN,NaN,City of Industry,NaN,NaN,41069.0,6.8
52650,62117,CN,UNK,2018-01-01,NaN,NaN,True,Retail Power Marketer,0.0,0.0,NaN,220.0,0.0,0.0,220.0,NaN,NaN,NaN,27300.0,NaN,NaN,27300.0,NaN,NaN,220.0,NaN,220.0,NaN,NaN,SFE Energy Texas,NaN,NaN,NaN,NaN


In [300]:
import numpy as np

idx_cols = [
    'utility_id_eia',
    'state',
    'nerc_region',
    'report_date',
]

recognized_nerc_regions = [
    'ASCC',
    'ECAR',
    'ERCOT',
    'FRCC',
    'HICC',
    'MAAC',
    'MAIN',
    'MAPP',
    'MRO',
    'NPCC',
    'RFC',
    'SERC',
    'SPP',
    'TRE',
    'WECC',
    'WSCC',  # pre-2002 version of WECC
    'MISO',  # unclear whether technically a regional entity, but lots of entries
    'AK',  # Alaska
    'HI',  # Hawaii
    'VI',  # Virgin Islands
    'GU',  # Guam
    'PR',  # Puerto Rico
    'AS',  # American Samoa
    'UNK',
]

nerc_spellcheck = {
    'GUSTAVUSAK': 'AK',
    'ERCTO': 'ERCOT',
    'RFO': 'RFC',
    'RF': 'RFC',
    'SSP': 'SPP',
    'VACAR': 'SERC',  # VACAR is a subregion of SERC
    'GATEWAY': 'SERC',  # GATEWAY is a subregion of SERC
    'TERR': 'GU',
}

Version with APPLY

In [365]:
%%time

# Create an index list without nerc_region
idx_no_nerc = idx_cols.copy()
if 'nerc_region' in idx_no_nerc:
    idx_no_nerc.remove('nerc_region')

# Split raw df into primary keys plus nerc region and other value cols
nerc_df = test_df[idx_cols]
other_df = test_df.drop('nerc_region', axis=1).set_index(idx_no_nerc)

# Backfill the utility with strange numeric nerc region
nerc_df.loc[nerc_df.utility_id_eia == 55959, 'nerc_region'] = 'MRO'

# Make all values upper-case
# Replace all NA values with UNK
# Make nerc values into lists to see how many separate values are stuffed into one row (ex: 'SPP & ERCOT' --> ['SPP', 'ERCOT'])
nerc_df = (
    nerc_df.assign(
        nerc_region=(lambda x: (
            x.nerc_region
            .str.upper()
            .fillna('UNK')
            .str.findall(r'[A-Z]+')))
    )
)

# Record a list of the reported nerc regions not included in the recognized regions list (these eventually become UNK)
nerc_col = nerc_df['nerc_region'].tolist()
nerc_list = list(set([item for sublist in nerc_col for item in sublist]))
non_nerc_list = [nerc_entity for nerc_entity in nerc_list if nerc_entity not in recognized_nerc_regions + list(nerc_spellcheck.keys())]

def remove_unk_multiples(entity_list):
    if ((len(entity_list)> 1) & ('UNK' in entity_list)):
        entity_list.remove('UNK')
    return entity_list


# Go through the nerc regions, spellcheck errors, delete those that aren't recognized, and piece them back together
# (with _ separator if more than one recognized)
nerc_df['nerc_region'] = (
    nerc_df['nerc_region']
    .apply(lambda x:[i if i not in nerc_spellcheck.keys() else nerc_spellcheck[i] for i in x])
    .apply(lambda x: sorted([i if i in recognized_nerc_regions else 'UNK' for i in x]))
    .apply(lambda x: remove_unk_multiples(x))
    .str.join('_')
)

full_df = pd.merge(nerc_df, other_df, on=idx_no_nerc)

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


CPU times: user 739 ms, sys: 49 ms, total: 788 ms
Wall time: 827 ms


Version with STACK

In [80]:
%%time

# Create an index list without nerc_region
idx_no_nerc = idx_cols.copy()
if 'nerc_region' in idx_no_nerc:
    idx_no_nerc.remove('nerc_region')

# Split raw df into primary keys plus nerc region and other value cols
nerc_df = test_df[idx_cols]
other_df = test_df.drop('nerc_region', axis=1).set_index(idx_no_nerc)

# Backfill the utility with strange numeric nerc region
nerc_df.loc[nerc_df.utility_id_eia == 55959, 'nerc_region'] = 'MRO'

# Make all values upper-case
# Replace all NA values with UNK
# Make nerc values into lists to see how many separate values are stuffed into one row (ex: SPP & ERCOT)
nerc_df = (
    nerc_df.assign(
        nerc_region=(lambda x: (
            x.nerc_region
            .str.upper()
            .fillna('UNK')
            .str.findall(r'[A-Z]+')
            .str.join('_')))
    )
)

nerc_df_expanded = (
    nerc_df
    .set_index(idx_no_nerc)
    .stack()
    .str.split(',', expand=True)
    .stack()
    .unstack(-2)
    .reset_index(-1, drop=True)
    .replace(nerc_spellcheck)
    .reset_index()
)

# list of regions that are turned to UNK:
no_nerc_df = nerc_df_expanded[~nerc_df_expanded['nerc_region'].isin(recognized_nerc_regions)].reset_index()
no_nerc_list = no_nerc_df.groupby(idx_no_nerc)['nerc_region'].transform(lambda x: '_'.join(x.astype('str'))).drop_duplicates().tolist()
#logger.info(f"The following reported nerc regions: {non_nerc_list} are being converted to UNK")

nerc_df_expanded['nerc_region'] = (
    nerc_df_expanded['nerc_region']
    .apply(lambda x: x if x in recognized_nerc_regions else 'UNK')
)

nerc_df_compressed = (
    nerc_df_expanded
    .drop_duplicates()
    .sort_values('nerc_region')
    .groupby(idx_no_nerc, observed=True).agg({'nerc_region': lambda x: '_'.join(x)})
)

CPU times: user 1.09 s, sys: 30.5 ms, total: 1.12 s
Wall time: 1.51 s


# Preliminary Data Wrangling
Once all of the data is loaded and looks like it's in good shape, do any initial wrangling that's specific to this particular analysis. This should mostly make use of the higher level functions which were defined above. If this step takes a while, don't be shy about producing `logging` outputs.

In [8]:
idx_cols = [
    'utility_id_eia',
    'state',
    'balancing_authority_code_eia',
    'report_date',
]

CUSTOMER_CLASSES = [
    "commercial",
    "dircnct",  # previously direct_connection
    "industrial",
    "other",
    "residential",
    "total",
    "transportation"
]

# 1495 w/0 ba
# 1489 w/ ba

In [10]:
df = eia_transformed_dfs['non_net_metering_eia861'].copy()
df2 = eia861_raw_dfs['non_net_metering_eia861'].copy()
df = pudl.helpers.organize_cols(df, idx_cols+['utility_name_eia','customer_class', 'tech_class'])
#df[df['7'].notnull()]
df

,utility_id_eia,state,balancing_authority_code_eia,report_date,utility_name_eia,customer_class,tech_class,backup_capacity_mw,capacity_mw,current_flow_type,generators_number,utility_owned_capacity_mw
0,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,combturb,NaN,NaN,NaN,1.0,NaN
1,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,fcell,NaN,NaN,NaN,1.0,NaN
2,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,hydro,NaN,NaN,NaN,1.0,NaN
3,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,ice,NaN,NaN,NaN,1.0,NaN
4,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,other,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
90775,99999,WY,PACE,2018-01-01,Adjustment 2018,transportation,pv,NaN,NaN,NaN,NaN,NaN
90776,99999,WY,PACE,2018-01-01,Adjustment 2018,transportation,steam,NaN,NaN,NaN,NaN,NaN
90777,99999,WY,PACE,2018-01-01,Adjustment 2018,transportation,storage,NaN,NaN,NaN,NaN,NaN
90778,99999,WY,PACE,2018-01-01,Adjustment 2018,transportation,total,NaN,NaN,NaN,NaN,NaN


In [14]:
df2.columns.tolist()

['backup_capacity_mw',
 'balancing_authority_code_eia',
 'commercial_combturb_capacity_mw',
 'commercial_fcell_capacity_mw',
 'commercial_hydro_capacity_mw',
 'commercial_ice_capacity_mw',
 'commercial_other_capacity_mw',
 'commercial_pv_capacity_mw',
 'commercial_storage_capacity_mw',
 'commercial_total_capacity_mw',
 'commercial_wind_capacity_mw',
 'dircnct_combturb_capacity_mw',
 'dircnct_fcell_capacity_mw',
 'dircnct_hydro_capacity_mw',
 'dircnct_ice_capacity_mw',
 'dircnct_other_capacity_mw',
 'dircnct_pv_capacity_mw',
 'dircnct_steam_capacity_mw',
 'dircnct_storage_capacity_mw',
 'dircnct_total_capacity_mw',
 'dircnct_wind_capacity_mw',
 'generators_number',
 'industrial_combturb_capacity_mw',
 'industrial_fcell_capacity_mw',
 'industrial_hydro_capacity_mw',
 'industrial_ice_capacity_mw',
 'industrial_other_capacity_mw',
 'industrial_pv_capacity_mw',
 'industrial_steam_capacity_mw',
 'industrial_storage_capacity_mw',
 'industrial_total_capacity_mw',
 'industrial_wind_capacity_mw'

In [17]:
# # Define base comparison df 
# eia_base_df = (
#     pudl_out.utils_eia860()
#     #[['report_date', 'utility_id_eia', 'utility_id_pudl', 'utility_name_eia', 'state']]
#     .assign(utility_id_eia=lambda x: x.utility_id_eia.astype('Int64'))
# )
# # Shorten df
# #df = df[['utility_id_eia', 'state', 'utility_name_eia', 'report_date']]

# # Merge dfs together 
# df_merge = (
#     pd.merge(eia_base_df, df, on=['utility_id_eia', 'state', 'report_date'], how='outer', suffixes=('_base', '_861'))
#     .set_index(['utility_id_eia', 'report_date'])
#     .assign(utility_name_eia_base=lambda x: x.utility_name_eia_base.astype('string'),
#             utility_name_eia_861=lambda x: x.utility_name_eia_861.astype('string'))
# )
# df_merge

## Fuzzy Match Columns

In [85]:
from fuzzywuzzy import fuzz

def fuzzy_match(df, col1, col2):
    # Define base comparison df 
    eia_base_df = (
        pudl_out.utils_eia860()
        #[['report_date', 'utility_id_eia', 'utility_id_pudl', 'utility_name_eia', 'state']]
        .assign(utility_id_eia=lambda x: x.utility_id_eia.astype('Int64'))
    )
    # Shorten df
    #df = df[['utility_id_eia', 'state', 'utility_name_eia', 'report_date']]
    
    # Merge dfs together 
    df_merge = (
        pd.merge(eia_base_df, df, on=['utility_id_eia', 'state'], how='outer', suffixes=('_base', '_861'))
        .set_index(['utility_id_eia', 'state'])
        .assign(utility_name_eia_base=lambda x: x.utility_name_eia_base.astype('string'),
                utility_name_eia_861=lambda x: x.utility_name_eia_861.astype('string'))
    )
    
    # Only run where both columns for utility name have values
    df_merge_no_null = (
        df_merge.loc[
            (df_merge[col1].notnull()) 
            & (df_merge[col2].notnull())].copy()
    )
    
    # Fuzzy match
    df_merge_no_null['fuzzy'] = (
        df_merge_no_null.apply(lambda x: fuzz.ratio(x.utility_name_eia_base, x.new_parent),axis=1)
    )
#     df_merge_no_null = (
#         df_merge_no_null.assign(fuzzy=lambda x: fuzz.ratio(x[col1], x[col2])))
    
    #df_merge_no_null.loc[:,'fuzzy'] = fuzzy_series
    df_merge_no_null = df_merge_no_null.sort_values('fuzzy')
    
    return df_merge_no_null

# DSM Stuff

In [37]:
dsm = eia861_raw_dfs['demand_side_management_eia861'].copy()
#dsm.columns.tolist()

In [50]:
idx_cols = ['utility_id_eia', 'state', 'ba_code', 'report_year']

dsm_test = dsm[['utility_id_eia', 'state', 'ba_code', 'report_year', 
                'residential_annual_load_management_energy_effects ', 
                'commercial_annual_load_management_energy_effects ', 
                'industrial_annual_load_management_energy_effects ',
                'transportation_annual_load_management_energy_effects ',
                'total_annual_load_management_energy_effects ',
]]

dr_test = raw_dr_eia861[['utility_id_eia', 'state', 'balancing_authority_code_eia', 'report_year',
                         'residential_energy_savings_mwh',
                         'commercial_energy_savings_mwh',
                         'industrial_energy_savings_mwh',
                         'transportation_energy_savings_mwh',
                         'total_energy_savings_mwh'
]].rename(columns={'balancing_authority_code_eia': 'ba_code'})

In [84]:
merge = pd.merge(dsm_test, dr_test, on=idx_cols, how='outer')
merge = pudl.helpers.oob_to_nan(merge, 
                       ['residential_annual_load_management_energy_effects ', 
                        'commercial_annual_load_management_energy_effects ', 
                        'industrial_annual_load_management_energy_effects ',
                        'transportation_annual_load_management_energy_effects ',
                        'residential_energy_savings_mwh',
                        'commercial_energy_savings_mwh',
                        'industrial_energy_savings_mwh',
                        'transportation_energy_savings_mwh',
                        'total_energy_savings_mwh',
                        'total_annual_load_management_energy_effects '])

In [71]:
dr_col = 'residential_energy_savings_mwh'
dsm_col = 'residential_annual_load_management_energy_effects '

In [36]:
merge = merge.loc[merge['report_year'].isin(range(2012,2014))]
merge = merge.loc[merge[dr_col].notna()]
#merge = merge.loc[merge[dsm_col].notna()]


NameError: name 'merge' is not defined

In [34]:
#test.plot.scatter('residential_annual_load_management_energy_effects ', 'residential_energy_savings_mwh')

In [33]:
#raw_dr_eia861.columns.tolist()

# Data Analysis and Visualization
* Now that you've got the required data in a usable form, you can tell the story of your analysis through a mix of visualizations, and further data wrangling steps.
* This narrative should be readable, with figures that have titles, legends, and labeled axes as appropriate so others can understand what you're showing them.
* The code should be concise and make use of the parameters and functions which you've defined above when possible. Functions should contain comprehensible chunks of work that make sense as one step in the story of the analysis.